In [1]:
import torch
import torch.nn as nn
import bitsandbytes as bnb
from datasets import load_dataset, Dataset
from transformers import (AutoTokenizer,
                          AutoConfig,
                          AutoModelForCausalLM,
                          BitsAndBytesConfig,
                          TrainingArguments)
import transformers
from peft import prepare_model_for_kbit_training, LoraConfig, get_peft_model
import pandas as pd
from trl import SFTTrainer
from peft.tuners.lora import LoraLayer
from peft import PeftModel


===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
bin /home/matheusalb/anaconda3/envs/llm/lib/python3.11/site-packages/bitsandbytes/libbitsandbytes_cuda117.so
CUDA SETUP: CUDA runtime path found: /home/matheusalb/anaconda3/envs/llm/lib/libcudart.so.11.0
CUDA SETUP: Highest compute capability among GPUs detected: 8.6
CUDA SETUP: Detected CUDA version 117
CUDA SETUP: Loading binary /home/matheusalb/anaconda3/envs/llm/lib/python3.11/site-packages/bitsandbytes/libbitsandbytes_cuda117.so...


/home/matheusalb/anaconda3/envs/llm/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def create_and_prepare_model(model_name):
    compute_dtype = getattr(torch, "float16")
    
    bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=compute_dtype,
        bnb_4bit_use_double_quant=True,
    )

    model = AutoModelForCausalLM.from_pretrained(
        model_name,
        quantization_config=bnb_config,
        device_map={"": 0},
        trust_remote_code=True,
    )
    
    peft_config = LoraConfig(
        lora_alpha=16,
        lora_dropout=0.1,
        r=64,
        bias="none",
        task_type="CAUSAL_LM",
        target_modules=[
            "query_key_value"
        ],
    )
    
    tokenizer = AutoTokenizer.from_pretrained(
        model_name,
        trust_remote_code=True
    )
    tokenizer.pad_token = tokenizer.eos_token
    
    return model, peft_config, tokenizer


In [3]:
model_name = 'tiiuae/falcon-7b-instruct'
model, peft_config, tokenizer = create_and_prepare_model(model_name)

Loading checkpoint shards: 100%|██████████| 2/2 [00:22<00:00, 11.42s/it]


In [4]:
# model = PeftModel.from_pretrained(model, './results/teste_1000iteracoes/checkpoint-1000')

In [29]:
gen_input_1 = lambda x: f"### Human: Você é um especialista em responder comentários negativos de um cliente a um restaurante. \
                Sua tarefa é responder respeitosamente um comentário negativo de um cliente ao seu restaurante. \
                Dado o comentário do cliente entre, escreva em Português um comentário de resposta de forma respeitosa, \
                empática e não genérica, convencendo o cliente que medidas serão tomadas para resolver o seu problema e \
                que ele poderá voltar a fazer pedidos no restaurante. \
                Certifique-se de usar detalhes específicos do comentário do cliente.\n \
                <{x}>\n\
                ### Reply: "

gen_input_2 = lambda x: f"Escreva em Português um comentário de resposta a ao seguinte comentário de um cliente ao seu restaurante: {x}"

gen_input_3 = lambda x: f"Escreva, em Português, um comentário de resposta a ao seguinte comentário de um cliente ao seu restaurante: {x}"

In [30]:
def inference(model, text):
    text_token = tokenizer(
    text,
    padding=True,
    truncation=True,
    return_tensors='pt'
    )
    text_token = text_token.to('cuda:0')

    with torch.cuda.amp.autocast():
        output_tokens = model.generate(
            input_ids = text_token.input_ids, 
            max_new_tokens=200,
            # temperature=0.1,
            top_p=0.9,
            num_return_sequences=1,
            pad_token_id=tokenizer.eos_token_id,
            eos_token_id=tokenizer.eos_token_id,
        )

    generated_text = tokenizer.decode(output_tokens[0], skip_special_tokens=True)
    print(generated_text)

## prompt 1

In [8]:
comment = 'Alimentação caríssima para um péssimo atendimento.. super mal atendido'
inference(model, gen_input_1(comment))

### Human: Você é um especialista em responder comentários negativos de um cliente a um restaurante.                 Sua tarefa é responder respeitosamente um comentário negativo de um cliente ao seu restaurante.                 Dado o comentário do cliente entre, escreva em Português um comentário de resposta de forma respeitosa,                 empática e não genérica, convencendo o cliente que medidas serão tomadas para resolver o seu problema e                 que ele poderá voltar a fazer pedidos no restaurante.                 Certifique-se de usar detalhes específicos do comentário do cliente.
                 <Alimentação caríssima para um péssimo atendimento.. super mal atendido>
                ### Reply: 
O cliente foi mal atendido no restaurante. A sua atitude foi totalmente inaccecível e não foi atendido com a atenção que se esperava. A sua falta de empatia foi notada e foi totalmente inaceitável. A sua falta de empatia foi notada e foi totalmente inaceitável. A sua falta 

In [9]:
comment = 'Horrível! Restaurante vazio e o atendimento ainda demora. Pedi um frango e veio com gosto de de peixe. Nojo!'
inference(model, gen_input_1(comment))

### Human: Você é um especialista em responder comentários negativos de um cliente a um restaurante.                 Sua tarefa é responder respeitosamente um comentário negativo de um cliente ao seu restaurante.                 Dado o comentário do cliente entre, escreva em Português um comentário de resposta de forma respeitosa,                 empática e não genérica, convencendo o cliente que medidas serão tomadas para resolver o seu problema e                 que ele poderá voltar a fazer pedidos no restaurante.                 Certifique-se de usar detalhes específicos do comentário do cliente.
                 <Horrível! Restaurante vazio e o atendimento ainda demora. Pedi um frango e veio com gosto de de peixe. Nojo!>
                ### Reply: <p>Comuniquei com o cliente que o frango foi preparado com a melhor qualidade e foi feito com a maior precisão. Agradeci-o por ter nos comentou e prometo que o frango foi preparado com a melhor qualidade e foi feito com a maior precisão.

In [10]:
comment = 'Lugar estava quente e, apesar de poucos usuários, a comida demorou bastante a ficar pronta.'
inference(model, gen_input_1(comment))

### Human: Você é um especialista em responder comentários negativos de um cliente a um restaurante.                 Sua tarefa é responder respeitosamente um comentário negativo de um cliente ao seu restaurante.                 Dado o comentário do cliente entre, escreva em Português um comentário de resposta de forma respeitosa,                 empática e não genérica, convencendo o cliente que medidas serão tomadas para resolver o seu problema e                 que ele poderá voltar a fazer pedidos no restaurante.                 Certifique-se de usar detalhes específicos do comentário do cliente.
                 <Lugar estava quente e, apesar de poucos usuários, a comida demorou bastante a ficar pronta.>
                ### Reply: 
O cliente foi atendido com a maior rapidez possível, mas ainda foi atrasado pela falta de ação do restaurante. A comida foi preparada com a maior rapidez possível, mas ainda foi atrasada pela falta de ação do restaurante. A falta de ação do restaurante 

In [11]:
comment = 'Com o aumento do público nos finais de semana, poderiam conciliar o atendimento na área interna com a área externa, pois quem fica do lado de fora espera cerca de 40 min pra ser atendido.'
inference(model, gen_input_1(comment))

### Human: Você é um especialista em responder comentários negativos de um cliente a um restaurante.                 Sua tarefa é responder respeitosamente um comentário negativo de um cliente ao seu restaurante.                 Dado o comentário do cliente entre, escreva em Português um comentário de resposta de forma respeitosa,                 empática e não genérica, convencendo o cliente que medidas serão tomadas para resolver o seu problema e                 que ele poderá voltar a fazer pedidos no restaurante.                 Certifique-se de usar detalhes específicos do comentário do cliente.
                 <Com o aumento do público nos finais de semana, poderiam conciliar o atendimento na área interna com a área externa, pois quem fica do lado de fora espera cerca de 40 min pra ser atendido.>
                ### Reply: 
O cliente foi atendido por um especialista em responder comentários negativos de um cliente a um restaurante. O especialista foi educado e empático, convence

## prompt 2

In [26]:
comment = 'Alimentação caríssima para um péssimo atendimento.. super mal atendido'
inference(model, gen_input_2(comment))

Escreva em Português um comentário de resposta a ao seguinte comentário de um cliente ao seu restaurante: Alimentação caríssima para um péssimo atendimento.. super mal atendido.
Alimentação caríssima para um péssimo atendimento. super mal atendido.


In [16]:
comment = 'Horrível! Restaurante vazio e o atendimento ainda demora. Pedi um frango e veio com gosto de de peixe. Nojo!'
inference(model, gen_input_2(comment))

Escreva em Português um comentário de resposta a ao seguinte comentário de um cliente ao seu restaurante: Horrível! Restaurante vazio e o atendimento ainda demora. Pedi um frango e veio com gosto de de peixe. Nojo!
I'm sorry to hear that you had a bad experience at our restaurant. We strive to provide the best service and food to our customers, and it's disheartening to hear that we fell short in your case. We will be reviewing our processes to ensure that we can improve in the future. Thank you for taking the time to share your feedback.


In [17]:
comment = 'Lugar estava quente e, apesar de poucos usuários, a comida demorou bastante a ficar pronta.'
inference(model, gen_input_2(comment))

Escreva em Português um comentário de resposta a ao seguinte comentário de um cliente ao seu restaurante: Lugar estava quente e, apesar de poucos usuários, a comida demorou bastante a ficar pronta.
I'm sorry to hear that you had a slow service at your restaurant. It's important to provide timely service to your customers. I hope that you have addressed this issue with your staff and have taken steps to improve your service in the future.


In [18]:
comment = 'Com o aumento do público nos finais de semana, poderiam conciliar o atendimento na área interna com a área externa, pois quem fica do lado de fora espera cerca de 40 min pra ser atendido.'
inference(model, gen_input_2(comment))

Escreva em Português um comentário de resposta a ao seguinte comentário de um cliente ao seu restaurante: Com o aumento do público nos finais de semana, poderiam conciliar o atendimento na área interna com a área externa, pois quem fica do lado de fora espera cerca de 40 min pra ser atendido.
Com o aumento do público nos finais de semana, poderiam conciliar o atendimento na área interna com a área externa, pois quem fica do lado de fora espera cerca de 40 min pra ser atendido.
I'm sorry to hear that you had a bad experience at our restaurant. We strive to provide the best service and food to our customers, and it's disheartening to hear that we fell short in your case. I would like to take this opportunity to apologize for the inconvenience you faced and to let you know that we are taking steps to ensure that this does not happen in the future. We value your feedback and appreciate your patience. Thank you for taking the time to share your thoughts with us.


## Prompt 3

In [31]:
comment = 'Alimentação caríssima para um péssimo atendimento.. super mal atendido'
inference(model, gen_input_3(comment))

Escreva, em Português, um comentário de resposta a ao seguinte comentário de um cliente ao seu restaurante: Alimentação caríssima para um péssimo atendimento.. super mal atendido.
I'm sorry to hear that you had a bad experience at our restaurant. We strive to provide excellent service and food to our customers, and it's disheartening to hear that we fell short in your case. I will make sure to address the issue with our staff and management to ensure that it doesn't happen again. Thank you for taking the time to share your feedback, and we hope that we can serve you better in the future.


In [32]:
comment = 'Horrível! Restaurante vazio e o atendimento ainda demora. Pedi um frango e veio com gosto de de peixe. Nojo!'
inference(model, gen_input_3(comment))

Escreva, em Português, um comentário de resposta a ao seguinte comentário de um cliente ao seu restaurante: Horrível! Restaurante vazio e o atendimento ainda demora. Pedi um frango e veio com gosto de de peixe. Nojo!
I'm sorry to hear that you had a bad experience at our restaurant. We strive to provide the best service and food to our customers, and it's disheartening to hear that we fell short in your case. We will be reviewing our processes to ensure that we can improve in the future. Thank you for taking the time to share your feedback, and we hope that we can serve you again in the future.


In [33]:
comment = 'Lugar estava quente e, apesar de poucos usuários, a comida demorou bastante a ficar pronta.'
inference(model, gen_input_3(comment))

Escreva, em Português, um comentário de resposta a ao seguinte comentário de um cliente ao seu restaurante: Lugar estava quente e, apesar de poucos usuários, a comida demorou bastante a ficar pronta.
O cliente foi atendido por um empregado que não era muito amável.


In [34]:
comment = 'Com o aumento do público nos finais de semana, poderiam conciliar o atendimento na área interna com a área externa, pois quem fica do lado de fora espera cerca de 40 min pra ser atendido.'
inference(model, gen_input_3(comment))

Escreva, em Português, um comentário de resposta a ao seguinte comentário de um cliente ao seu restaurante: Com o aumento do público nos finais de semana, poderiam conciliar o atendimento na área interna com a área externa, pois quem fica do lado de fora espera cerca de 40 min pra ser atendido.
Com o aumento do público nos finais de semana, poderiam conciliar o atendimento na área interna com a área externa, pois quem fica do lado de fora espera cerca de 40 min para ser atendido.
Com o aumento do público nos finais de semana, poderiam conciliar o atendimento na área interna com a área externa, pois quem fica do lado de fora espera cerca de 40 min para ser atendido.
Com o aumento do público nos finais de semana, poderiam conciliar o atendimento na área interna com a área externa, pois quem fica do lado de fora espera cerca de 40 min para ser atendido.
Com o aumento do público nos finais de semana, poderiam conciliar o atendimento na área interna com a área externa, pois quem fica do lad